# Week 1: Data Setup and Exploratory Data Analysis

## Objectives
1. Load Spotify tracks dataset and Top 100 hits dataset
2. Merge datasets and create HIT/NON-HIT labels
3. Handle class imbalance
4. Perform exploratory data analysis
5. Visualize feature distributions
6. Save processed dataset

## Timeline
**Week 1 (Nov 18-24)** - Due: Nov 24 (Checkpoint)

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from fuzzywuzzy import fuzz
from tqdm.auto import tqdm

# Settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [2]:
# Set up paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
raw_data_dir = data_dir / 'raw'
processed_data_dir = data_dir / 'processed'
figures_dir = project_root / 'figures'

# Create directories if they don't exist
processed_data_dir.mkdir(parents=True, exist_ok=True)
figures_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Figures directory: {figures_dir}")

Project root: C:\Users\FR33STYLE\FeatureBeats
Data directory: C:\Users\FR33STYLE\FeatureBeats\data
Figures directory: C:\Users\FR33STYLE\FeatureBeats\figures


## 1. Load Datasets

We'll load:
1. **Main tracks dataset** - All Spotify tracks with audio features
2. **Top 100 dataset** - Chart-topping hits (these will be labeled as HITS)

In [3]:
# Load main tracks dataset
print("Loading main tracks dataset...")
tracks_file = raw_data_dir / 'tracks.csv'

if not tracks_file.exists():
    raise FileNotFoundError(
        f"❌ tracks.csv not found at {tracks_file}\n"
        "Please download the Spotify Tracks dataset from Kaggle and place it in data/raw/"
    )

tracks_df = pd.read_csv(tracks_file)
print(f"✅ Loaded {len(tracks_df):,} tracks")
print(f"   Columns: {list(tracks_df.columns)}")
print(f"\nFirst few rows:")
tracks_df.head()

Loading main tracks dataset...
✅ Loaded 169,909 tracks
   Columns: ['acousticness', 'artists', 'danceability', 'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo', 'valence', 'year']

First few rows:


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [4]:
# Load Top 100 / Billboard hits dataset
print("Loading Top 100 hits dataset...")
hits_file = raw_data_dir / 'top100_tracks.csv'

if not hits_file.exists():
    raise FileNotFoundError(
        f"❌ top100_tracks.csv not found at {hits_file}\n"
        "Please download a Top 100/Billboard dataset from Kaggle and place it in data/raw/"
    )

hits_df = pd.read_csv(hits_file)
print(f"✅ Loaded {len(hits_df):,} hit songs")
print(f"   Columns: {list(hits_df.columns)}")
print(f"\nFirst few rows:")
hits_df.head()

Loading Top 100 hits dataset...
✅ Loaded 2,400 hit songs
   Columns: ['playlist_url', 'year', 'track_id', 'track_name', 'track_popularity', 'album', 'artist_id', 'artist_name', 'artist_genres', 'artist_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

First few rows:


,playlist_url,year,track_id,track_name,track_popularity,album,artist_id,artist_name,artist_genres,artist_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000,6naxalmIoLFWR0siv8dnQQ,Oops!...I Did It Again,81,Oops!... I Did It Again,26dSoYclwsYLMAKD3tpOr4,Britney Spears,"['dance pop', 'pop']",81,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,211160,4
1,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,83,Enema Of The State,6FBDaR13swtiWwGhX1WQsP,blink-182,"['alternative metal', 'modern rock', 'pop punk...",79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,167067,4
2,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000,3y4LxiYMgDl4RethdzpmNe,Breathe,66,Breathe,25NQNriVT2YbSW80ILRWJa,Faith Hill,"['contemporary country', 'country', 'country d...",62,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,250547,4
3,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000,0v1XpBHnsbkCn7iJ9Ucr1l,It's My Life,81,Crush,58lV9VcRSjABbAbfWS6skp,Bon Jovi,"['glam metal', 'rock']",79,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,224493,4
4,https://open.spotify.com/playlist/37i9dQZF1DWU...,2000,62bOmKYxYg7dhrC6gH9vFn,Bye Bye Bye,75,No Strings Attached,6Ff53KvcvAj5U7Z1vojB5o,*NSYNC,"['boy band', 'dance pop', 'pop']",70,0.610,0.926,8,-4.843,0,0.0479,0.0310,0.001200,0.0821,0.861,172.638,200400,4


## 2. Data Preprocessing

### 2.1 Identify Required Columns

We need to identify which columns contain:
- Track name
- Artist name
- Audio features (danceability, energy, etc.)
- Year/release date
- (Optional) Track ID

In [5]:
# Inspect tracks dataset structure
print("Tracks dataset info:")
print(tracks_df.info())
print("\nSample row:")
print(tracks_df.iloc[0])

Tracks dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169909 entries, 0 to 169908
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      169909 non-null  float64
 1   artists           169909 non-null  object 
 2   danceability      169909 non-null  float64
 3   duration_ms       169909 non-null  int64  
 4   energy            169909 non-null  float64
 5   explicit          169909 non-null  int64  
 6   id                169909 non-null  object 
 7   instrumentalness  169909 non-null  float64
 8   key               169909 non-null  int64  
 9   liveness          169909 non-null  float64
 10  loudness          169909 non-null  float64
 11  mode              169909 non-null  int64  
 12  name              169909 non-null  object 
 13  popularity        169909 non-null  int64  
 14  release_date      169909 non-null  object 
 15  speechiness       169909 non-null  float64
 16 

In [6]:
# Inspect hits dataset structure
print("Hits dataset info:")
print(hits_df.info())
print("\nSample row:")
print(hits_df.iloc[0])

Hits dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   playlist_url       2400 non-null   object 
 1   year               2400 non-null   int64  
 2   track_id           2400 non-null   object 
 3   track_name         2400 non-null   object 
 4   track_popularity   2400 non-null   int64  
 5   album              2400 non-null   object 
 6   artist_id          2400 non-null   object 
 7   artist_name        2400 non-null   object 
 8   artist_genres      2400 non-null   object 
 9   artist_popularity  2400 non-null   int64  
 10  danceability       2400 non-null   float64
 11  energy             2400 non-null   float64
 12  key                2400 non-null   int64  
 13  loudness           2400 non-null   float64
 14  mode               2400 non-null   int64  
 15  speechiness        2400 non-null   float64
 16  acous

### 2.2 Standardize Column Names

**Note:** The column names below are examples. You may need to adjust them based on your actual dataset.

Common Spotify audio features:
- `danceability`
- `energy`
- `key`
- `loudness`
- `mode`
- `speechiness`
- `acousticness`
- `instrumentalness`
- `liveness`
- `valence`
- `tempo`
- `duration_ms`

In [7]:
# Define audio features to use (adjust based on your dataset)
audio_features = [
    'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 
    'valence', 'tempo'
]

# Check which features are available
available_features = [f for f in audio_features if f in tracks_df.columns]
missing_features = [f for f in audio_features if f not in tracks_df.columns]

print(f"✅ Available features ({len(available_features)}): {available_features}")
if missing_features:
    print(f"⚠️  Missing features ({len(missing_features)}): {missing_features}")

# Use only available features
audio_features = available_features

✅ Available features (9): ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


### 2.3 Filter Data by Year Range

Focus on a specific time period (e.g., 2010-2020) to ensure temporal relevance.

In [8]:
# Define year range
YEAR_START = 2010
YEAR_END = 2020

# Try to find year column (adjust column name as needed)
year_col = None
for col in ['year', 'release_year', 'release_date', 'album_release_date']:
    if col in tracks_df.columns:
        year_col = col
        break

if year_col:
    print(f"Using '{year_col}' column for year filtering")
    
    # Extract year if it's a date string
    if tracks_df[year_col].dtype == 'object':
        tracks_df['year'] = pd.to_datetime(tracks_df[year_col], errors='coerce').dt.year
    else:
        tracks_df['year'] = tracks_df[year_col]
    
    # Filter by year range
    tracks_filtered = tracks_df[
        (tracks_df['year'] >= YEAR_START) & 
        (tracks_df['year'] <= YEAR_END)
    ].copy()
    
    print(f"✅ Filtered to {YEAR_START}-{YEAR_END}: {len(tracks_filtered):,} tracks")
else:
    print("⚠️  No year column found. Using all data.")
    tracks_filtered = tracks_df.copy()
    tracks_filtered['year'] = 2015  # Default year

print(f"\nYear distribution:")
print(tracks_filtered['year'].value_counts().sort_index())

Using 'year' column for year filtering
✅ Filtered to 2010-2020: 21,656 tracks

Year distribution:
year
2010    2000
2011    2000
2012    2000
2013    2000
2014    2000
2015    1931
2016    1969
2017    2000
2018    2000
2019    2000
2020    1756
Name: count, dtype: int64


## 3. Create HIT Labels

We'll match songs from the Top 100 dataset with the main tracks dataset to create labels:
- **HIT (1)** - Song appears in Top 100
- **NON-HIT (0)** - Song does not appear in Top 100

### Matching Strategy
1. **Best:** Match by Spotify track ID (if available)
2. **Good:** Exact match on (track name + artist name)
3. **Fallback:** Fuzzy match on (track name + artist name)

In [9]:
# Prepare hits dataset
# Adjust column names based on your dataset
print("Preparing hits dataset for matching...")

# Try to find track name and artist columns in hits dataset
track_name_col = None
artist_col = None

for col in ['track_name', 'title', 'song', 'name']:
    if col in hits_df.columns:
        track_name_col = col
        break

for col in ['artist', 'artist_name', 'artists']:
    if col in hits_df.columns:
        artist_col = col
        break

if not track_name_col or not artist_col:
    print("⚠️  Could not auto-detect column names. Please set them manually:")
    print(f"   Available columns: {list(hits_df.columns)}")
    # Uncomment and set manually if needed:
    # track_name_col = 'your_track_column'
    # artist_col = 'your_artist_column'
else:
    print(f"✅ Using track column: '{track_name_col}'")
    print(f"✅ Using artist column: '{artist_col}'")

# Standardize hits dataset
hits_standardized = hits_df[[track_name_col, artist_col]].copy()
hits_standardized.columns = ['track_name', 'artist']

# Clean text for better matching
hits_standardized['track_name'] = hits_standardized['track_name'].str.lower().str.strip()
hits_standardized['artist'] = hits_standardized['artist'].str.lower().str.strip()

print(f"\nPrepared {len(hits_standardized):,} hit songs for matching")

Preparing hits dataset for matching...
✅ Using track column: 'track_name'
✅ Using artist column: 'artist_name'

Prepared 2,400 hit songs for matching


In [10]:
# Prepare main tracks for matching
print("Preparing tracks dataset for matching...")

# Find track name and artist columns in tracks dataset
track_col_tracks = None
artist_col_tracks = None

for col in ['track_name', 'title', 'song', 'name']:
    if col in tracks_filtered.columns:
        track_col_tracks = col
        break

for col in ['artist', 'artist_name', 'artists']:
    if col in tracks_filtered.columns:
        artist_col_tracks = col
        break

if not track_col_tracks or not artist_col_tracks:
    print("⚠️  Could not auto-detect column names. Please set them manually:")
    print(f"   Available columns: {list(tracks_filtered.columns)}")
else:
    print(f"✅ Using track column: '{track_col_tracks}'")
    print(f"✅ Using artist column: '{artist_col_tracks}'")

# Create standardized matching columns
tracks_filtered['track_clean'] = tracks_filtered[track_col_tracks].str.lower().str.strip()
tracks_filtered['artist_clean'] = tracks_filtered[artist_col_tracks].str.lower().str.strip()
tracks_filtered['match_key'] = tracks_filtered['track_clean'] + '___' + tracks_filtered['artist_clean']

# Create match key for hits
hits_standardized['match_key'] = hits_standardized['track_name'] + '___' + hits_standardized['artist']
hit_keys = set(hits_standardized['match_key'].unique())

print(f"Created {len(hit_keys):,} unique hit song identifiers")

Preparing tracks dataset for matching...
✅ Using track column: 'name'
✅ Using artist column: 'artists'
Created 2,286 unique hit song identifiers


In [11]:
# Create HIT labels using exact matching
print("Creating HIT labels...\n")

tracks_filtered['is_hit'] = tracks_filtered['match_key'].isin(hit_keys).astype(int)

# Check class balance
hit_counts = tracks_filtered['is_hit'].value_counts()
total = len(tracks_filtered)

print("Class Distribution:")
print(f"  NON-HIT (0): {hit_counts.get(0, 0):,} ({hit_counts.get(0, 0)/total*100:.2f}%)")
print(f"  HIT (1):     {hit_counts.get(1, 0):,} ({hit_counts.get(1, 0)/total*100:.2f}%)")
print(f"\n  Total tracks: {total:,}")
print(f"  Imbalance ratio: {hit_counts.get(0, 0) / max(hit_counts.get(1, 1), 1):.1f}:1")

if hit_counts.get(1, 0) < 50:
    print("\n⚠️  WARNING: Very few hits matched. You may need to:")
    print("   1. Check if column names are correct")
    print("   2. Use fuzzy matching (see next cell)")
    print("   3. Try a different hits dataset")

Creating HIT labels...

Class Distribution:
  NON-HIT (0): 21,656 (100.00%)
  HIT (1):     0 (0.00%)

  Total tracks: 21,656
  Imbalance ratio: 21656.0:1

⚠️  WARNING: Very few hits matched. You may need to:
   1. Check if column names are correct
   2. Use fuzzy matching (see next cell)
   3. Try a different hits dataset


### Optional: Fuzzy Matching (if exact matching found too few hits)

In [ ]:
# Uncomment and run this cell if you need fuzzy matching

def fuzzy_match_song(track, artist, hits_df, threshold=85):
    """Find best fuzzy match for a song in hits dataset (robust to NaNs)."""
    # If both are missing, don't even try
    if pd.isna(track) and pd.isna(artist):
        return False

    # Safely convert inputs to lowercase strings
    track_str = "" if pd.isna(track) else str(track).lower().strip()
    artist_str = "" if pd.isna(artist) else str(artist).lower().strip()

    best_score = 0
    for _, hit in hits_df.iterrows():
        # Safely convert hits dataframe values too
        hit_track = "" if pd.isna(hit['track_name']) else str(hit['track_name']).lower().strip()
        hit_artist = "" if pd.isna(hit['artist']) else str(hit['artist']).lower().strip()

        # Score based on both track name and artist
        track_score = fuzz.ratio(track_str, hit_track)
        artist_score = fuzz.ratio(artist_str, hit_artist)
        combined_score = (track_score + artist_score) / 2

        if combined_score > best_score:
            best_score = combined_score

    return best_score >= threshold


# Apply fuzzy matching (this may take a while for large datasets)
print("Applying fuzzy matching... (this may take several minutes)")

# Only check songs that weren't already matched
non_matched = tracks_filtered[tracks_filtered['is_hit'] == 0].copy()

# Sample if dataset is too large
# if len(non_matched) > 10000:
#     print("Sampling 10,000 tracks for fuzzy matching (dataset too large)")
#     non_matched = non_matched.sample(10000, random_state=RANDOM_SEED)

fuzzy_hits = []
for idx, row in tqdm(non_matched.iterrows(), total=len(non_matched)):
    if fuzzy_match_song(row[track_col_tracks], row[artist_col_tracks], hits_standardized):
        fuzzy_hits.append(idx)

# Update labels
tracks_filtered.loc[fuzzy_hits, 'is_hit'] = 1

print(f"\n✅ Found {len(fuzzy_hits)} additional hits using fuzzy matching")

# Show updated distribution
print("\nUpdated class distribution:")
print(tracks_filtered['is_hit'].value_counts())


Applying fuzzy matching... (this may take several minutes)


  0%|          | 0/21656 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## 4. Clean and Prepare Final Dataset

In [ ]:
# Select relevant columns for modeling
columns_to_keep = audio_features + ['is_hit', 'year']

# Add track info for reference (not for modeling)
if track_col_tracks in tracks_filtered.columns:
    columns_to_keep.append(track_col_tracks)
if artist_col_tracks in tracks_filtered.columns:
    columns_to_keep.append(artist_col_tracks)

# Create final dataset
df_final = tracks_filtered[columns_to_keep].copy()

# Handle missing values
print("Missing values per column:")
missing = df_final.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
    print("\nDropping rows with missing values...")
    df_final = df_final.dropna()
    print(f"✅ Removed {len(tracks_filtered) - len(df_final)} rows")
else:
    print("✅ No missing values found")

print(f"\nFinal dataset shape: {df_final.shape}")
print(f"Features: {audio_features}")
print(f"Target: is_hit")

In [ ]:
# Display final dataset info
print("Final dataset summary:")
df_final.info()
print("\nFirst few rows:")
df_final.head(10)

## 5. Exploratory Data Analysis

### 5.1 Basic Statistics

In [ ]:
# Summary statistics
print("Summary statistics for audio features:\n")
df_final[audio_features].describe()

In [ ]:
# Summary by class
print("Summary statistics by class:\n")
print("NON-HITS (0):")
print(df_final[df_final['is_hit'] == 0][audio_features].describe())
print("\n" + "="*80 + "\n")
print("HITS (1):")
print(df_final[df_final['is_hit'] == 1][audio_features].describe())

### 5.2 Visualizations

#### Distribution of Tracks by Year

In [ ]:
# Tracks by year
fig, ax = plt.subplots(figsize=(12, 6))

year_counts = df_final.groupby(['year', 'is_hit']).size().unstack(fill_value=0)
year_counts.plot(kind='bar', stacked=True, ax=ax, color=['#3498db', '#e74c3c'])

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Number of Tracks', fontsize=12)
ax.set_title('Distribution of Tracks by Year and Hit Status', fontsize=14, fontweight='bold')
ax.legend(['Non-Hit', 'Hit'], title='Class')
plt.xticks(rotation=45)
plt.tight_layout()

# Save figure
plt.savefig(figures_dir / 'tracks_by_year.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: tracks_by_year.png")

#### Feature Distributions: Hits vs Non-Hits

In [ ]:
# Box plots for each feature
n_features = len(audio_features)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten() if n_features > 1 else [axes]

for idx, feature in enumerate(audio_features):
    ax = axes[idx]
    
    # Create box plot
    data_to_plot = [
        df_final[df_final['is_hit'] == 0][feature],
        df_final[df_final['is_hit'] == 1][feature]
    ]
    
    bp = ax.boxplot(data_to_plot, labels=['Non-Hit', 'Hit'], 
                     patch_artist=True, showmeans=True)
    
    # Color the boxes
    bp['boxes'][0].set_facecolor('#3498db')
    bp['boxes'][1].set_facecolor('#e74c3c')
    
    ax.set_ylabel(feature.capitalize(), fontsize=11)
    ax.set_title(f'{feature.capitalize()} Distribution', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

# Hide extra subplots
for idx in range(n_features, len(axes)):
    axes[idx].set_visible(False)

plt.suptitle('Audio Feature Distributions: Hits vs Non-Hits', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()

# Save figure
plt.savefig(figures_dir / 'feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: feature_distributions.png")

#### Correlation Matrix

In [ ]:
# Correlation matrix
fig, ax = plt.subplots(figsize=(12, 10))

# Calculate correlation
corr_matrix = df_final[audio_features + ['is_hit']].corr()

# Create heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            ax=ax)

ax.set_title('Feature Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()

# Save figure
plt.savefig(figures_dir / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: correlation_matrix.png")
print("\nCorrelation with 'is_hit':")
print(corr_matrix['is_hit'].sort_values(ascending=False))

## 6. Save Processed Dataset

In [ ]:
# Save the processed dataset
output_file = processed_data_dir / 'hits_dataset.csv'
df_final.to_csv(output_file, index=False)

print(f"✅ Saved processed dataset to: {output_file}")
print(f"   Shape: {df_final.shape}")
print(f"   Features: {len(audio_features)}")
print(f"   Samples: {len(df_final):,}")
print(f"   Hits: {(df_final['is_hit'] == 1).sum():,}")
print(f"   Non-hits: {(df_final['is_hit'] == 0).sum():,}")

## 7. Key Findings Summary

### Data Overview
- **Total tracks**: 

In [ ]:
# Generate summary report
print("="*80)
print("WEEK 1 SUMMARY REPORT")
print("="*80)

print(f"\n1. DATASET STATISTICS")
print(f"   Total tracks: {len(df_final):,}")
print(f"   Hit songs: {(df_final['is_hit'] == 1).sum():,} ({(df_final['is_hit'] == 1).sum()/len(df_final)*100:.2f}%)")
print(f"   Non-hit songs: {(df_final['is_hit'] == 0).sum():,} ({(df_final['is_hit'] == 0).sum()/len(df_final)*100:.2f}%)")
print(f"   Year range: {df_final['year'].min():.0f} - {df_final['year'].max():.0f}")

print(f"\n2. CLASS IMBALANCE")
ratio = (df_final['is_hit'] == 0).sum() / max((df_final['is_hit'] == 1).sum(), 1)
print(f"   Imbalance ratio: {ratio:.1f}:1 (non-hits : hits)")
if ratio > 10:
    print(f"   ⚠️  SEVERE IMBALANCE - Will need class weighting or SMOTE")
elif ratio > 5:
    print(f"   ⚠️  MODERATE IMBALANCE - Will use class weighting")
else:
    print(f"   ✅ Manageable imbalance")

print(f"\n3. FEATURE CORRELATIONS WITH HIT STATUS")
print("   Top 5 positive correlations:")
correlations = corr_matrix['is_hit'].drop('is_hit').sort_values(ascending=False)
for i, (feature, corr) in enumerate(correlations.head().items(), 1):
    print(f"   {i}. {feature}: {corr:+.4f}")

print(f"\n   Top 5 negative correlations:")
for i, (feature, corr) in enumerate(correlations.tail().items(), 1):
    print(f"   {i}. {feature}: {corr:+.4f}")

print(f"\n4. NEXT STEPS (Week 2)")
print(f"   → Build logistic regression baseline")
print(f"   → Implement class weighting")
print(f"   → Evaluate with appropriate metrics (F1, Recall, PR-AUC)")
print(f"   → Interpret coefficients")

print(f"\n5. FILES CREATED")
print(f"   → {output_file}")
print(f"   → {figures_dir / 'tracks_by_year.png'}")
print(f"   → {figures_dir / 'feature_distributions.png'}")
print(f"   → {figures_dir / 'correlation_matrix.png'}")

print("\n" + "="*80)
print("✅ WEEK 1 COMPLETE - Ready for modeling!")
print("="*80)

---

## ✅ Week 1 Deliverables Complete!

You now have:
1. ✅ Cleaned and labeled dataset
2. ✅ Understanding of class imbalance
3. ✅ Visualizations showing feature distributions
4. ✅ Correlation analysis

### For Checkpoint Presentation (Nov 24):
Use these figures:
- `tracks_by_year.png` - Show dataset scope
- `feature_distributions.png` - Show how hits differ from non-hits
- `correlation_matrix.png` - Highlight most correlated features

### Next Steps:
Proceed to **02_Week2_Baseline_Modeling.ipynb** to build the logistic regression baseline.

---